In [ ]:
import numpy as np
from numpy import math

from bokeh.layouts import column, row
from bokeh.models import ColumnDataSource, CustomJS, Button, RadioButtonGroup, Slider
from bokeh.models import CheckboxButtonGroup, Panel, Tabs, RadioGroup, CheckboxGroup, Div
from bokeh.plotting import figure
from bokeh.plotting import figure, show, output_file

In [ ]:
# With Young interferention only, no difraction

max_x = 30  # Délka stínítka
var_x = 1  #  Rozptyl normálního rozdělení
slitdist = 1 # Vzdálenost štěrbin 
slitwidth = 1 # Šířka štěrbin
color = "red" # Barva teček
source = ColumnDataSource(data=dict(x=[], y=[], colors=[]), tags=[dict(max_x=max_x, color=color)])

# Screen with points
p = figure(
    title="Screen of the double slit experiment",
    tools="",         # (de)activate tools    
    sizing_mode="stretch_width", 
#    max_width=1000, 
    width=800,
    height=400,
    x_axis_label="x [cm]",
    y_axis_label=None,
    x_range=(0,max_x),
    y_range=(0, 1)
)

# Delete y-axis ticks
p.yaxis.minor_tick_in = 0
p.yaxis.minor_tick_out = 0
p.yaxis.major_tick_in = 0
p.yaxis.major_tick_out = 0
p.yaxis.major_label_text_color = "white"

p.circle('x', 'y', source=source, size=7, fill_color='colors', line_color="black")

# Probability denstity function above the screen + histogram
pd = figure(title="Probability density",
            sizing_mode="stretch_width", 
            width=p.width, height=250, x_range=p.x_range, tools="")

# # Theoretical probability density functions
# amp = 1  # ------------> jak předávat hodnoty těchto konstant do js funkce youngPDF(x)?
# freq = 1
# x = np.linspace(0, max_x, 500)
# # Young double slit interference
# young_pdf = amp*np.cos(freq*x)**2

# # Normal distribution (mean, var_x)
# # normal_pdf = 1
# source_pd = ColumnDataSource(data=dict(x=x, y=young_pdf), 
#                              tags=[dict(var_x=var_x)])  
# pd.line('x', 'y', source=source_pd, line_width=2, line_color="navy")

# Theoretical probability density functions
amp = 1  # ------------> jak předávat hodnoty těchto konstant do js funkce youngPDF(x)?
freq = 1
# x = np.linspace(0, max_x, 500)
# Young double slit interference
# young_pdf = amp*np.cos(freq*x)**2

# Normal distribution (mean, var_x)
# normal_pdf = 1
source_pd = ColumnDataSource(data=dict(x=[], y=[]), 
                             tags=[dict(var_x=var_x)])  
pd.line('x', 'y', source=source_pd, line_width=2, line_color="navy")

# Histogram -- young double slit
Nbins = 60 # number of bins
binwidth = max_x/Nbins
hist, bins = np.histogram([], range=(0, max_x), bins=Nbins) 
# hist = rel. četnosti, bins = array dělicích bodů binů, counts = abs četnosti
source_hist = ColumnDataSource(data=dict(hist=[0 for _ in range(Nbins)], 
                                         bins=[binwidth/2 + i*binwidth for i in range(Nbins)], # dělení vystředované
                                         counts=[0 for _ in range(Nbins)]), # array abs četností
                               tags=[dict(binwidth=binwidth)])  
pd.vbar(top="hist", x="bins", width=binwidth, source=source_hist, alpha=0.6, color="skyblue")

# # Tabs CLASS x QUANTUM x WAVES
# tab_class = Panel(child=layout, title="Classical bullets")
# tab_quantum = Panel(child=layout, title="Quantum objects")
# tab_waves = Panel(child=layout, title="Waves (water waves)")

# Buttons ----------------------------------------------------------------------
# Add 1 particle
button = Button(label="Add 1 particle", button_type="success")

# Send 100 particles at once
button100 = Button(label="Send 100 particles", button_type="success")

# Reset the screen
resetbutton = Button(label="Reset", button_type="success")

# Continuous stream of particles
streamtoggle = Button(label="Send continuous stream", button_type="success")

# Change distribution
LABELS = ["Uniform", "Normal", "Young"]
button_change_dist = RadioButtonGroup(labels=LABELS, active=0)

# Block/open slits
LABELS = ["Slit 1", "Slit 2"]
button_choose_slit = CheckboxButtonGroup(labels=LABELS, active=[0, 1])

# Detector
button_detector = Button(label="Detector", button_type="default")

#Slider
slider_slitdist = Slider(start=0, end=10, value=1, step=0.1, title="Slit distance")
slider_slitwidth = Slider(start=0, end=5, value=0.1, step=0.1, title="Slit width")
slider_wavelength = Slider(start=0, end=5, value=0.1, step=0.1, title="Wavelength [nm]")

# Text -- energy label
energy = Div(text="This is some text.", width=200, height=50)

# Choose particle colour
LABELS_color = ["Red", "Blue", "Green"]
color_button = RadioGroup(labels=LABELS_color, active=0)

# Keep previous measurements after set up is changed
button_keep = CheckboxGroup(labels=["Keep previous measurements"], active=[])

# -----------------------------------------------------------------------------
# JS functions definition

# Def function of normal distribution with parameters mean, var (to be read from sliders)
js_normalPDF = """
    function randn(mean, variance) { // Box Muller Transform
      let u = 0, v = 0;
      
      while(u === 0) u = Math.random(); //Converting [0,1) to (0,1)
      while(v === 0) v = Math.random();
      
      let num = Math.sqrt( -2.0 * Math.log( u ) ) * Math.cos( 2 * Math.PI * v );
      //num = num*source_pd.tags[0]['var_x'] + (source.tags[0]['max_x']/2 - source_pd.tags[0]['slitdist']); // Shift mean  0 -> max_x/2; var = 1 -> var_x
      num = num * variance + mean
      
      if (num < 0) {return randn()}  // resample between 0 and max_x (abandon values out of screen)
      else if (source.tags[0]['max_x'] < num) {return randn();}
      else {return num;}
    }
 """

# Def function of Young double slit interference distribution
js_youngPDF = """
    function youngPDF(x) { // PDF of Young interferention
        const amp = 1;
        const freq = 1;
        return amp*Math.cos(freq*x)**2
        
//      return source_pd.tags[0].amp*Math.cos(freq*x)**2
    }
    
    let minI = 0;
    let N = 100;
    
    function youngCDF(maxI) { // CDF of Young interferention
      let integral = 0;
      let delta = (maxI - minI)/N
      for (let i = 0; i < N; i++) {
          integral += youngPDF(delta/2 + i*delta)*delta;
      } 
      return integral
    }
    
    const sample = [];    // array of divistion points of the x-axis (up to max_x, step 0.01)
    for (let i = 0; i < source.tags[0].max_x; i += 0.01) {
      sample.push(i);
    }
    const sample_y = sample.map(youngCDF);  // youngCDF function values = the division points of the y-axis
    
    // Normalization of PDF to the length of shade (max_x)
    const h = sample_y[sample_y.length-1] - sample_y[0];

    let index;
    function youngInverseCDF(t) { // inverse CDF of Young interferention
        for (let i = 0; i < sample_y.length; i++) {
            if (sample_y[i] > t) {
            index = i;
            break;
            }
        }
        const x2 = sample[index];
        const x1 = sample[index - 1];
        const y1 = sample_y[index - 1]
        const c = (x2 - x1)/(sample_y[index] + y1);
        return c*t - c*y1 + x1;
    }
"""

# Add one particle 
js_send = """
    function send(u, renormalize=true) {
        if (u!=false) { //nezapo49t8v8 částice při zavřených štěrbinách
            source.data.x.push(u)
            source.data.y.push(Math.random())
            source.data.colors.push(source.tags[0].color) // Read particle colour from source
            source.change.emit()  // update the data source with local changes

            // add to histogram
            let j = Math.floor((u)/s_hist.tags[0].binwidth);  // index of bin where a particle with coord. new_x is added
            s_hist.data.counts[j]++;
            if (renormalize) {normalizehist()}; 
        }
        
    }
    
    // normalize histogram
    function normalizehist() {
       // normalization; number of particles = source.data.x.length
       let totalarea = source.data.x.length*s_hist.tags[0].binwidth; // total number of particles: source.data.x.length
        for (let i = 0; i < s_hist.data.hist.length; i++) {
            s_hist.data.hist[i] = s_hist.data.counts[i]/totalarea;
        }
        s_hist.change.emit();
    }
"""

# Generator of a random number from a given distribution (uniform / normal(max_x/2, var_x) / young)
js_generator = """
    function generator() {
        //console.log(slit.active)
        if (slit.active.length==2){
           if (detector.button_type == "primary") {
               //console.log('detektor')
               if (Math.random()<0.5) {
                   return randn(source.tags[0].max_x/2 - slider_slitdist.value, source_pd.tags[0].var_x)
               }
               else {
                   return randn(source.tags[0].max_x/2 + slider_slitdist.value, source_pd.tags[0].var_x)
               }
           }
           else {
               return youngInverseCDF(h*Math.random())
           }
        } 
        else if (slit.active.length==1){
            if (slit.active[0]==0 ) {
                return randn(source.tags[0].max_x/2 - slider_slitdist.value, source_pd.tags[0].var_x)
            }
            else {
                return randn(source.tags[0].max_x/2 + slider_slitdist.value, source_pd.tags[0].var_x)
            }
        }
        else {
            return false
        }
    }
"""

js_reset = """
    function reset() {
        // reset data in source
        source.data.x = []
        source.data.y = []
        source.data.colors = []
        source.change.emit()

        // reset histogram
        for (let i = 0; i < s_hist.data.hist.length; i++) {
                s_hist.data.hist[i] = 0;
                s_hist.data.counts[i] = 0;
                }
        s_hist.change.emit()

        // reset pdf
        source_pd.data.x = []
        source_pd.data.y = []
        source_pd.change.emit()
    }
"""

#-------------------------------------------------------------------------
# JS CALLBACKS
js_functions = js_normalPDF + js_youngPDF + js_send + js_generator + js_reset

js_cb_objects = dict(source=source, source_pd=source_pd, s_hist=source_hist, 
                     streamtoggle=streamtoggle, 
                     selected=button_change_dist, slit=button_choose_slit, detector=button_detector, 
                     slider_slitdist=slider_slitdist, slider_slitwidth=slider_slitwidth, slider_wavelength=slider_wavelength, 
                     color_button=color_button, energy=energy)

# # # Theoretical pdf -- switch according to distribution buttons -- Young without diffraction
callback_Teorpdf = CustomJS(args=js_cb_objects, code="""
    
    const slitdist = slider_slitdist.value
    const slitwidth = slider_slitwidth.value
    
    energy.text = "Energy: <b>" + slitdist + "</b> [eV]"
    
    
    
    // normalization  constant for Young interference on the interval (0; max_x)
    // const Norm = 2/(source.tags[0].max_x + Math.sin(source.tags[0].max_x))  
    const Norm = 2/(source.tags[0].max_x + Math.sin(source.tags[0].max_x)*Math.cos(source.tags[0].max_x))
    
    if (source_pd.data.x.length==0) {
        let step = source.tags[0].max_x/500;
        for (let i = 0; i <= 500; i++) {
             source_pd.data.x.push(i*step)}
    }
    
    const x = source_pd.data.x
        
    let y;
    
    if (slit.active.length==2) {
        if (detector.button_type == "primary") { // Sum of both slits normal distributions  
            y = Array.from(x, (x) => (1/(Math.sqrt(2*Math.PI)*source_pd.tags[0].var_x))/2 * 
                                      Math.exp(-0.5*((x - (source.tags[0].max_x/2 - slitdist))/source_pd.tags[0].var_x)**2) + 
                                     (1/(Math.sqrt(2*Math.PI)*source_pd.tags[0].var_x))/2 * 
                                      Math.exp(-0.5*((x - (source.tags[0].max_x/2 + slitdist))/source_pd.tags[0].var_x)**2)   )
        }
        else{ // Young interference
              y = Array.from(x, (x) => Norm * (Math.cos(x))**2  )
        }
    }
    else if (slit.active.length==1) { 
    
        if (slit.active[0]==0) { // Left slit normal distribution
            y = Array.from(x, (x) =>  (1/(Math.sqrt(2*Math.PI)*source_pd.tags[0].var_x)) * 
                                      Math.exp(-0.5*((x - (source.tags[0].max_x/2 - slitdist))/source_pd.tags[0].var_x)**2) )
        }
        else { // Right slit normal distribution
            y = Array.from(x, (x) =>  (1/(Math.sqrt(2*Math.PI)*source_pd.tags[0].var_x)) * 
                                      Math.exp(-0.5*((x - (source.tags[0].max_x/2 + slitdist))/source_pd.tags[0].var_x)**2) )
        }
    }
    else { // Both slits blocked, nothing
        y = Array.from(x, (x) => 0 )
    }
           

    source_pd.data = { x, y }
    source_pd.change.emit()
""")

# # Theoretical pdf -- switch according to distribution buttons -- Young + diffraction
# callback_Teorpdf = CustomJS(args=js_cb_objects, code="""

#     const slitdist = slider_slitdist.value
#     const slitwidth = slider_slitwidth.value

#     const x = source_pd.data.x
#     let y;
    
#     if (slit.active.length==2) {
#         if (detector.button_type == "primary") { // Sum of both slits normal distributions  
#             y = Array.from(x, (x) => Math.exp(-0.5*((x - (source.tags[0].max_x/2 - slitdist))/source_pd.tags[0].var_x)**2) + 
#                                      Math.exp(-0.5*((x - (source.tags[0].max_x/2 + slitdist))/source_pd.tags[0].var_x)**2)   )
#         }
#         else if (0) { // Young interference
#               y = Array.from(x, (x) => (1/(source.tags[0].max_x/2 + Math.sin(source.tags[0].max_x))) * 
#                                        (Math.cos(x))**2  )
#         }
#         else { // Young interference + diffraction
#             y = Array.from(x, (x) => (1/(source.tags[0].max_x/2 + Math.sin(source.tags[0].max_x))) * 
#                                        ((Math.sin((x - source.tags[0].max_x/2)*slitwidth))**2 * 
#                                        (Math.cos((x - source.tags[0].max_x/2)*slitdist))**2) / 
#                                        ((x-source.tags[0].max_x/2)*slitwidth)**2  )
#         }
#     }
#     else if (slit.active.length==1) { 
    
#         if (slit.active[0]==0) { // Left slit normal distribution
#             y = Array.from(x, (x) =>  Math.exp(-0.5*((x - (source.tags[0].max_x/2 - slitdist))/source_pd.tags[0].var_x)**2) )
#         }
#         else { // Right slit normal distribution
#             y = Array.from(x, (x) =>  Math.exp(-0.5*((x - (source.tags[0].max_x/2 + slitdist))/source_pd.tags[0].var_x)**2) )
#         }
#     }
#     else { // Both slits blocked, nothing
#         y = Array.from(x, (x) => 0 )
#     }
           

#     source_pd.data = { x, y }
#     source_pd.change.emit()
# """)


callback = CustomJS(args=js_cb_objects, code=js_functions+"""
    send(generator())
    """)

callback100 = CustomJS(args=js_cb_objects, code=js_functions+"""   
    for (let i = 0; i < 100; i++) {
        send(generator(), false)
    }
    normalizehist()
    // console.log(selected.active)
    """)

callback_reset = CustomJS(args=js_cb_objects, code=js_functions+"""  
    reset()
    """)

callback_stream = CustomJS(args=js_cb_objects, code=js_functions+"""
    function sendparticle() {
        send(generator())
    }
      
    if (streamtoggle.button_type == "success") {
        const interval = setInterval(sendparticle, 1000);
        source.interval = interval;
        streamtoggle.button_type = "danger";
        streamtoggle.label = "Stop stream";
        
    } else {
        clearInterval(source.interval);
        streamtoggle.button_type = "success";
        streamtoggle.label = "Send continuous stream";
    }
""")

# Block/open slits, change button color
# --------- nefunguje, jak se přepíná type CheckBoxGroup? // chyba v bokehu, mus9 se aktualizovat
# https://panel.holoviz.org/reference/widgets/CheckButtonGroup.html
callback_choose_slit = CustomJS(args=js_cb_objects, code=js_functions+ """     
    // console.log(slit_button.value)
""")

# Switch detector active/no detector, change button color
callback_detector = CustomJS(args=js_cb_objects, code=js_functions+"""

    // Change detector colour  
    if (detector.button_type == "default") {
        detector.button_type = "primary";
        detector.label = "Detector active";
        
    } else {
        detector.button_type = "default";
        detector.label = "No detector present";
    }
    
    // If Keep not checked, reset after switching on detector
    if (button_keep.active==[0]) {
        
    }
""")

# Choose partice colour
callback_choose_color = CustomJS(args=js_cb_objects, code=js_functions+"""
    if (color_button.active==0) {
       source.tags[0].color = "red"
       console.log("red")
    }
    else if (color_button.active==1) {
        source.tags[0].color = "#ffc857"
        console.log("blue")
    }
    else {
        source.tags[0].color = "#2b9720"
        console.log("green")
    }
    console.log(color_button.active)
   // source.change.emit()
""")
# -------------------------------------------------------------------------------
# Callbacks to objects

button.js_on_event('button_click', callback, callback_Teorpdf)
button100.js_on_event('button_click', callback100, callback_Teorpdf)
resetbutton.js_on_event('button_click', callback_reset)
streamtoggle.js_on_event('button_click', callback_stream, callback_Teorpdf)
# button_change_dist.js_on_click('button_click', callback_)
button_choose_slit.js_on_event('button_click', callback_choose_slit, callback_Teorpdf)
button_detector.js_on_event('button_click', callback_detector, callback_Teorpdf)

slider_slitdist.js_on_change('value', callback_Teorpdf)
slider_slitwidth.js_on_change('value', callback_Teorpdf)
slider_wavelength.js_on_change('value', callback_Teorpdf)
color_button.js_on_click(callback_choose_color)

#buttons_layout = column(button, button100, resetbutton)
#show(row(p, buttons_layout))
layout = row(column(pd, p), 
         column(button, button100, resetbutton, streamtoggle, 
               button_choose_slit, button_detector, #  button_change_dist, 
#                 slider_amp, slider_freq, 
                slider_slitdist, slider_slitwidth, slider_wavelength, 
               color_button, button_keep, energy))
show(layout)
# Tabs(tabs=[tab_class, tab_quantum, tab_waves])

In [ ]:
# Tabs in a panel // to be used later"

from bokeh.io import show
from bokeh.models import Panel, Tabs
from bokeh.plotting import figure

p1 = figure(width=300, height=300)
p1.circle([1, 2, 3, 4, 5], [6, 7, 2, 4, 5], size=20, color="navy", alpha=0.5)
tab1 = Panel(child=p1, title="circle")

p2 = figure(width=300, height=300)
p2.line([1, 2, 3, 4, 5], [6, 7, 2, 4, 5], line_width=3, color="navy", alpha=0.5)
tab2 = Panel(child=p2, title="line")

show(Tabs(tabs=[tab1, tab2]))

In [ ]:
# LaTeX in Bokeh 3.0.0

from bokeh.io import show
from bokeh.models import Slider

slider = Slider(start=0, end=10, value=1, step=.1, title=r"$$\delta \text{ (damping factor, 1/s)}$$")

show(slider)

In [ ]:
from bokeh.io import show
from bokeh.models import CheckboxButtonGroup, CustomJS

LABELS = ["Option 1", "Option 2", "Option 3"]

checkbox_button_group = CheckboxButtonGroup(labels=LABELS, active=[0, 1])
checkbox_button_group.js_on_event("button_click", CustomJS(args=dict(btn=checkbox_button_group), code="""
    console.log('checkbox_button_group: active=' + btn.active, this.toString())
"""))

show(checkbox_button_group)

In [ ]:
# Get standalone html -- for Bokeh 3.0.0 only

from bokeh.plotting import figure
from bokeh.resources import CDN
from bokeh.embed import file_html

plot = figure()
plot.circle([1,2], [3,4])

html = file_html(plot, CDN, "my plot")

In [ ]:
from bokeh.plotting import figure, output_file, show

# Create data
x = [1, 2, 3, 4, 5]
y = [6, 7, 2, 4, 5]

# Specify output file
output_file("scatter_plot.html")

# Create a figure
p = figure(title="Scatter Plot", x_axis_label='X', y_axis_label='Y')

# Add scatter glyphs
p.circle(x, y, size=10, color='navy', alpha=0.5)

# Show the plot
show(p)